In [ ]:
import numpy as np
from datetime import datetime 
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.quantization
from torch.utils.tensorboard import SummaryWriter
import torchvision
import matplotlib.pyplot as plt
import time
import collections
from functools import partial
from mnist_model import ConvNet
from utils import *

In [ ]:
DEVICE = 'cpu'

# parameters
RANDOM_SEED = 42
LEARNING_RATE = 0.001
BATCH_SIZE = 10000
num_workers = 10

IMG_SIZE = 32
N_CLASSES = 10

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])
valid_dataset = datasets.MNIST(root='./data', train=False, transform=transform)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)
test_loader = DataLoader(dataset=valid_dataset, batch_size=1, shuffle=True, num_workers=num_workers, pin_memory=True)

In [ ]:
model = ConvNet(10)#.to(DEVICE)
model.load_state_dict(torch.load("mnist-convnet.pth", map_location=torch.device(DEVICE)))
capture = model.eval()

In [ ]:
get_accuracy(model, valid_loader, device=DEVICE)

In [ ]:
%debug

In [ ]:
weights, biases = get_weights_biases(model)

In [ ]:
np.array(([weight.max().detach().numpy() for weight in weights]))

In [ ]:
np.array(([weight.min().detach().numpy() for weight in weights]))

In [ ]:
np.array(([bias.max().detach().numpy() for bias in biases]))

In [ ]:
np.array(([bias.min().detach().numpy() for bias in biases]))

In [ ]:
activations = collections.defaultdict(lambda: None)
def save_activation(name, mod, inp, out):
    activations[name] = out # don't detach or move to CPU here
    
logged_layers = 0
for name, module in model.named_modules():
    if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
        module.register_forward_hook(partial(save_activation, name))
        logged_layers += 1

In [ ]:
plt.hist(np.maximum(activations['conv1'],0).flatten(), bins=20)
plt.yscale('log')

In [ ]:
np.percentile(np.maximum(activations['conv1'],0), 99.9)

In [ ]:
scaling_factor = 3
scaled_weights = [weight/scaling_factor for weight in weights]
scaled_biases = [bias/scaling_factor for bias in biases]

In [ ]:
# model.conv1.weight = nn.Parameter(model.conv1.weight*scaling_factor)
# model.conv1.bias = nn.Parameter(model.conv1.bias*scaling_factor)
model.conv2.weight = nn.Parameter(model.conv2.weight/1.5)
model.conv2.bias = nn.Parameter(model.conv2.bias/1.5)
model.conv3.weight = nn.Parameter(model.conv3.weight/1.5)
model.conv3.bias = nn.Parameter(model.conv3.bias/1.5)
model.fc1.weight = nn.Parameter(model.fc1.weight/scaling_factor)
model.fc1.bias = nn.Parameter(model.fc1.bias/scaling_factor)

In [ ]:
get_accuracy(model, valid_loader, device='cpu')

In [ ]:
#model.fc1.weight

In [ ]:
len(weights)

## Plot weights

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

weights = list(model.parameters())
weights = [weight.flatten() for weight in weights]
weights = torch.cat(weights)

capture = plt.hist(weights.cpu().detach().numpy(), bins=200)

## Model parameters

In [ ]:
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

In [ ]:
model.bn1.running_mean

# Quantization

## Example activation

In [ ]:
model.eval()
X, y_true = next(iter(test_loader))
X = X.to(DEVICE)
Y = y_true.to(DEVICE)
Y_hat = model(X)[0]
plt.plot(Y_hat.cpu().detach().numpy().flatten())


In [ ]:
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

In [ ]:
model.conv1.weight